In [1]:
Macro_state_path = "/export/scratch/for_haochong/DATA/macro_states/merged_macro_data.csv"
PV_data_path="/nfs/synof01e/intern/haochong/DATA/basedata/base_20000103_20171229_v240821"
import pandas as pd
import numpy as np

In [2]:
pv_data=pd.read_csv(PV_data_path,sep="|")
macro_state_data=pd.read_csv(Macro_state_path)

In [3]:
print(pv_data.head())
print(macro_state_data.head())
# INSCEPT MACRO DATA range
print(macro_state_data["date"].min(),macro_state_data["date"].max())
# check nan value in macro data
print(macro_state_data.isnull().sum())
print(macro_state_data.loc[:,"SPX_return"])

                 bbid      date     open     high     low     close    volume  \
0  EQ0010615300001000  20000104  42.3750  43.0625  41.750  42.09375    240451   
1  EQ0010439500001000  20000104   9.8750   9.8750   9.750   9.81250     17600   
2  EQ0010177100001000  20000104  73.4375  75.2500  71.500  71.75000  21098710   
3  EQ0010750900001000  20000104  10.0000  10.2500   9.875   9.87500    118800   
4  EQ0010501900001000  20000104  28.5000  31.6250  28.000  29.50000    720052   

            cap   sharesout  sector  industry  subindustry  caj  top500  \
0  7.424495e+08    17638000   10011     20081          193  1.0       0   
1  1.872617e+08    19084000   10005     20041          117  1.0       0   
2  1.120094e+11  1561106048   10013     20106          668  1.0       1   
3  4.582395e+08    46404000   10008     20055          164  1.0       0   
4  3.497815e+08    11857000   10003     20016          337  1.0       1   

   top1000  top2000  top3000  
0        0        1        1  


In [4]:
# process the data to only keep `bbid,date,open,close,high,low,volume`
def process_data_into_feature(v):
    if 'bbid' in v.columns:
        v = v[['bbid', 'date', 'industry', 'open', 'close', 'high', 'low', 'volume','caj']]


    # log the number of nan in each column
    print('Number of nan in each column: ')
    print(v.isna().sum())

    #uniqe bbid before drop na
    bbid_before =  v['bbid'].unique()
    # find the bbids where in the row with this bbid has nan in any column

    bbid_drop = v[v.isna().any(axis=1)]['bbid'].unique()
    # drop the rows of these bbids
    v = v[~v['bbid'].isin(bbid_drop)]
    #uniqe bbid after drop na
    bbid_after =  v['bbid'].unique()
    # print bbid droped
    print('bbid droped: ', len(np.setdiff1d(bbid_before, bbid_after)))

    # find the bbid where there is 0 in column ['open', 'close', 'high', 'low', 'volume']
    bbid_drop = v[(v[['open', 'close', 'high', 'low', 'volume']]==0).any(axis=1)]['bbid'].unique()
    # drop the rows of these bbids
    v = v[~v['bbid'].isin(bbid_drop)]
    # print bbid droped
    print('bbid droped: ', len(bbid_drop))

    return v

In [5]:
pv_data = process_data_into_feature(pv_data)

Number of nan in each column: 
bbid           0
date           0
industry       0
open        7376
close          0
high           0
low            0
volume         0
caj            0
dtype: int64
bbid droped:  692
bbid droped:  44


In [6]:
def process_data_into_feature(df):
    """
    Process the input DataFrame to generate the required features.
    
    Parameters:
    df (pd.DataFrame): DataFrame with columns ["open", "close", "high", "low", "volume", "caj"]
    
    Returns:
    pd.DataFrame: DataFrame with processed features
    pd.Series: Original close series
    pd.Series: caj column
    """
    # Smooth the OHLC data by dividing by caj
    smoothed_open = df['open'] / df['caj']
    smoothed_close = df['close'] / df['caj']
    smoothed_high = df['high'] / df['caj']
    smoothed_low = df['low'] / df['caj']
    
    # Calculate close(t+1) - close(t) return
    close_return = (smoothed_close.shift(-1) - smoothed_close) / smoothed_close
    
    # Calculate open(t+1) - close(t) return
    open_close_return = (smoothed_open.shift(-1) - smoothed_close) / smoothed_close

    # foward fill the nan in the close_return and open_close_return to avoid calculation error
    close_return = close_return.ffill()
    open_close_return = open_close_return.ffill()
    
    # Calculate high/close ratio
    high_close_ratio_return = smoothed_high / smoothed_close -1
    
    # Calculate low/close ratio
    low_close_ratio_return = smoothed_low / smoothed_close -1

    # # calcluate the volume return 
    # volume_return = (df['volume'].shift(-1) - df['volume']) / df['volume']
    # volume_return = volume_return.ffill()
    
    # Combine all features into a new DataFrame
    df_features = pd.DataFrame({
        'close_return': close_return,
        'open_close_return': open_close_return,
        'high_close_ratio_return': high_close_ratio_return,
        'low_close_ratio_return': low_close_ratio_return,
        # 'volume_return': volume_return,
        "volume": df['volume']
    })
    
    return df_features, smoothed_close, df["open"].iloc[0], df['volume'],  df['caj']

def revert_feature_into_data(df_features, original_close, original_open, original_volume , caj_column):
    """
    Revert the processed features back to the original OHLC data.
    
    Parameters:
    df_features (pd.DataFrame): DataFrame with processed features
    original_close (pd.Series): Series with the original close values
    caj_column (pd.Series): Series with the caj values
    
    Returns:
    pd.DataFrame: DataFrame with original columns ["open", "close", "high", "low", "volume"]
    """
    # Reconstruct smoothed close values from the features
    # reconstructed the close from the fiorst close value and by accumulating the close_return
    # smoothed_close = ((original_close * (1 + df_features.iloc[:,0]).cumprod()).shift(1).fillna(original_close))/caj_column
    # reconstruct ht close from the orginal close price and the close return
    # print('orginal_close: ', original_close)
    # print('orginal_volume: ', original_volume)
    smoothed_close = (original_close.shift(1) * (1 + df_features.iloc[:,0]).shift(1)).fillna(original_close.iloc[0])
    # print(smoothed_close.shift(1),(df_features.iloc[:,1]+1).shift(1))
    smoothed_open = (smoothed_close.shift(1) * (df_features.iloc[:,1]+1).shift(1)).fillna(original_open/caj_column.iloc[0])
    smoothed_high = smoothed_close * (1 + df_features.iloc[:,2])
    smoothed_low = smoothed_close * (1 + df_features.iloc[:,3])

    # log calcualtion of the smoothed close step by step
    # print("1+close_return: ", (1 + df_features.iloc[:,0]))
    # print("cumprod: ", ((original_close * (1 + df_features.iloc[:,0]).cumprod()).shift(1).fillna(original_close))/caj_column)
    # # calcualte the nan in the smoothed_close
    # print("nan in smoothed_close: ", smoothed_close.isna().sum())
    # print(smoothed_close, smoothed_open, smoothed_high, smoothed_low) 
    
    # Unsmooth the OHLC data by multiplying by caj
    close = smoothed_close * caj_column
    open_ = smoothed_open * caj_column
    high = smoothed_high * caj_column
    low = smoothed_low * caj_column
    # revert the volume with volume return
    # volume = (original_volume * (1 + df_features.iloc[:,4]).cumprod()).shift(1).fillna(original_volume)
    
    # Combine reconstructed values into a new DataFrame
    df_original = pd.DataFrame({
        'open': open_,
        'close': close,
        'high': high,
        'low': low,
        'volume': original_volume
    })
    
    return df_original

def curate_df_features(df_features):
    """
    Curate the features to ensure high_close_ratio_return never < 0 
    and low_close_ratio_return never > 0 using positional indexing.
    
    Parameters:
    df_features (pd.DataFrame): DataFrame with columns ["close_return", "open_close_return", 
                                                        "high_close_ratio_return", 
                                                        "low_close_ratio_return", "volume_return"]
    
    Returns:
    pd.DataFrame: Curated DataFrame with adjusted features
    """
    # Correct values in the high_close_ratio_return column (position 2)
    df_features.iloc[:, 2] = df_features.iloc[:, 2].apply(lambda x: max(x, 0))
    
    # Correct values in the low_close_ratio_return column (position 3)
    df_features.iloc[:, 3] = df_features.iloc[:, 3].apply(lambda x: min(x, 0))
    
    return df_features

In [7]:
# Creating a sample DataFrame for testing
data = pd.DataFrame({
    'open': [100, 102, 104, 103, 105],
    'close': [101, 103, 102, 106, 108],
    'high': [102, 104, 105, 107, 109],
    'low': [99, 100, 101, 102, 103],
    'volume': [1000, 1500, 1200, 1300, 1400],
    'caj': [2, 2, 2, 2, 2]
})

print("Original Data:")
print(data.head())
# Process the data
processed_data, original_close,original_open, original_volume, caj = process_data_into_feature(data.copy())
# print(original_close,original_open)
print("Processed Data:")
print(processed_data)

# Revert the processed data
original_data_reconstructed = revert_feature_into_data(
    processed_data,
    original_close,
    original_open,
    original_volume,
    caj
)

# Print the reconstructed data
print("Reconstructed Data:")
print(original_data_reconstructed)

# Check if the reconstructed data is the same as the original data by equality
print(original_data_reconstructed.equals(data))

Original Data:
   open  close  high  low  volume  caj
0   100    101   102   99    1000    2
1   102    103   104  100    1500    2
2   104    102   105  101    1200    2
3   103    106   107  102    1300    2
4   105    108   109  103    1400    2
Processed Data:
   close_return  open_close_return  high_close_ratio_return  \
0      0.019802           0.009901                 0.009901   
1     -0.009709           0.009709                 0.009709   
2      0.039216           0.009804                 0.029412   
3      0.018868          -0.009434                 0.009434   
4      0.018868          -0.009434                 0.009259   

   low_close_ratio_return  volume  
0               -0.019802    1000  
1               -0.029126    1500  
2               -0.009804    1200  
3               -0.037736    1300  
4               -0.046296    1400  
Reconstructed Data:
    open  close   high    low  volume
0  100.0  101.0  102.0   99.0    1000
1  102.0  103.0  104.0  100.0    1500
2  104

In [8]:
pv_selected_columns=["date","bbid","industry","close","open","high","low","volume",'caj']
pv_data=pv_data[pv_selected_columns]
# process the date column in both to make sure they are in the same format
pv_data["date"]=pd.to_datetime(pv_data["date"],format="%Y%m%d")

In [ ]:
pv_data_copy=pv_data.copy()
pv_features, pv_close, pv_open, pv_volume, pv_caj = process_data_into_feature(pv_data_copy)

In [9]:
data_path="/home/haochong/timegan-pytorch/data/by_industry/"
# create the data_path if it does not exist
import os
if not os.path.exists(data_path):
    os.makedirs(data_path)
# merge the two dataframes with the same date
pv_selected_columns=["date","bbid","industry","close","open","high","low","volume","caj"]
pv_data=pv_data[pv_selected_columns]
# process the date column in both to make sure they are in the same format
pv_data["date"]=pd.to_datetime(pv_data["date"],format="%Y%m%d")
macro_state_data["date"]=pd.to_datetime(macro_state_data["date"],format="%Y-%m-%d")
# foward fill the nan in the macro_state_data
macro_state_data=macro_state_data.ffill()
# drop the nan in the macro_state_data
macro_state_data=macro_state_data.dropna()
# ensure no nan in the macro_state_data
assert macro_state_data.isnull().sum().sum()==0
# show date range of the pv_data and macro_state_data
print(pv_data["date"].min(),pv_data["date"].max())
print(macro_state_data["date"].min(),macro_state_data["date"].max())
# group the pv_data by subindusty and merge each group with the macro_state_data
pv_data_grouped=pv_data.groupby("industry")
for name,group in pv_data_grouped:

    # merge the two dataframes
    merged_data=pd.merge(group,macro_state_data,on="date",how="inner")
    print(merged_data.head())
    print(merged_data.columns)
    print(merged_data.shape)
    # assert no nan in the merged data
    assert merged_data.isnull().sum().sum()==0
    # save the merged data
    merged_data.to_csv(data_path+str(name)+".csv",index=False)
    # break

2000-01-04 00:00:00 2017-12-29 00:00:00
2001-01-01 00:00:00 2017-12-29 00:00:00
        date                bbid  industry    close     open     high  \
0 2001-01-02  EQ0010047900001000     20010  48.0000  47.3125  48.1875   
1 2001-01-02  EQ0010111700001000     20010  20.3750  21.3750  21.5625   
2 2001-01-02  EQ0010073000001000     20010  18.7500  18.8750  19.0000   
3 2001-01-02  EQ0010059700001000     20010  69.0000  71.4375  72.7500   
4 2001-01-02  EQ0010047100001000     20010  34.8125  35.0625  36.5000   

       low   volume       caj  A191RP1Q027SBEA  FEDFUNDS_return    VIX  \
0  46.5625  3119000  0.971228              1.3        -0.065625  29.99   
1  20.0000   532000  0.955088              1.3        -0.065625  29.99   
2  18.1875   577400  0.962283              1.3        -0.065625  29.99   
3  69.0000   261700  1.000000              1.3        -0.065625  29.99   
4  34.2500  2598000  0.321314              1.3        -0.065625  29.99   

   SPX_return  Crude_return  
0     